In [1]:
# fetch the original table from web-scrapping
import pandas as pd
import numpy as np
from datetime import datetime

# import and inspect the table
df_eso_fide_players = pd.read_csv('..\\eso_and_fide_players_ratings.csv')
df_eso_fide_players.head()

,ΑΑ,ΚΩΔΙΚΟΣ,FIDE,ΟΝΟΜΑ,Η/Γ,ΦΥΛΟ,ΤΙΤΛΟΣ,ΕΛΟ,ΠΑΡΤΙΔΕΣ,*,...,fide link,eso player history,eso player last 6 month games,fide name,std,rapid,blitz,federation,National Rank All,National Rank Active
0,1,36055,NaN,ΜΑΪΕΡ ΧΑΝΣ - ΚΡΙΣΤΙΑΝ - ΜΙΧΑΕΛ,01/1959,Α,NaN,2336,0.0,NaN,...,http://ratings.fide.com/card.phtml?event=,/eso/player/36055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,30932,4232984.0,ΔΡΑΚΟΠΟΥΛΟΣ ΠΟΛΥΧΡΟΝΗΣ - ΠΑΝΑΓΙΩΤΗΣ,05/1998,Α,NaN,1961,2.0,NaN,...,http://ratings.fide.com/card.phtml?event=4232984,/eso/player/30932,/eso/playergames/30932,Drakopoulos Polihronis,2114,2037,1977,GRE,305.0,178.0
2,3,29392,4231813.0,ΤΣΑΓΚΑΡΟΠΟΥΛΟΣ ΣΠΥΡΙΔΩΝ,02/1989,Α,ΒΚ,1949,2.0,NaN,...,http://ratings.fide.com/card.phtml?event=4231813,/eso/player/29392,/eso/playergames/29392,Tsagkaropoulos Spyridon,2068,1999,1991,GRE,410.0,226.0
3,4,9345,4204123.0,ΣΠΥΡΑΚΟΠΟΥΛΟΣ ΙΩΑΝΝΗΣ,12/1971,Α,ΟΜ,1947,0.0,NaN,...,http://ratings.fide.com/card.phtml?event=4204123,/eso/player/09345,NaN,Spirakopoulos Ioannis,2035,1998,2009,GRE,488.0,263.0
4,5,25836,4211421.0,ΛΙΑΣΚΟΣ ΛΕΩΝΙΔΑΣ,03/1993,Α,ΑΚ,1929,2.0,NaN,...,http://ratings.fide.com/card.phtml?event=4211421,/eso/player/25836,/eso/playergames/25836,Liaskos Leonidas,2007,2011,2087,GRE,574.0,299.0


In [2]:
print(df_eso_fide_players.columns)

Index(['ΑΑ', 'ΚΩΔΙΚΟΣ', 'FIDE', 'ΟΝΟΜΑ', 'Η/Γ', 'ΦΥΛΟ', 'ΤΙΤΛΟΣ', 'ΕΛΟ',
       'ΠΑΡΤΙΔΕΣ', '*', 'ΤΔ', 'fide link', 'eso player history',
       'eso player last 6 month games', 'fide name', 'std', 'rapid', 'blitz',
       'federation', 'National Rank All', 'National Rank Active'],
      dtype='object')


## Reforming and Renaming the table

In [18]:
# replace NaN values and othe empty values: 'Not rated', '-' with '0'
modified_players_list = df_eso_fide_players.fillna('0').replace(['Not rated', '-'], '0')

# replace gender Greek characters to English
modified_players_list = modified_players_list.replace(['Α','Θ'], ['M','F'])

# create a column with the Fide Id hyperlinked when using .style attribute
def make_clickable(url, value_to_apply_url):
    return '<a href="{}">{}</a>'.format(url,value_to_apply_url)

modified_players_list['FIDE'] = modified_players_list['FIDE'].astype(int)
modified_players_list['Fide ID'] = modified_players_list.apply(lambda col: make_clickable(col['fide link'], col['FIDE']) , axis=1)

# discard hyperlink columns
modified_players_list = pd.concat([modified_players_list.iloc[:, :11] , modified_players_list.iloc[:, 14:]], axis=1)


# transform integer values to int type
for col in modified_players_list.columns:
    try:
        modified_players_list[col] = modified_players_list[col].astype(int)
    except:
        pass

# renaming column index 
modified_players_list = modified_players_list.rename(columns= {
                            'ΑΑ': 'Club Rank National',
                            'ΚΩΔΙΚΟΣ': 'National ID',
                            'FIDE': 'Fide',
                            'ΟΝΟΜΑ': 'National Name',
                            'Η/Γ': 'Date of Birth' ,
                            'ΦΥΛΟ': 'Gender',
                            'ΤΙΤΛΟΣ': "National Title",
                            'ΕΛΟ': 'National Elo',
                            'ΠΑΡΤΙΔΕΣ': 'Games last 6 months',
                            '*': 'Outsider' ,
                            'ΤΔ': 'Eso Fee',
                            'fide name' : 'Fide Name',
                            'federation' : 'Federation',
                            })

# parse the date of birth and creare a new Age column 
modified_players_list['Birthday'] = pd.to_datetime('01/'+ modified_players_list['Date of Birth'], dayfirst=True)
modified_players_list['Age'] = modified_players_list['Birthday'].map(lambda x: datetime.today().year - x.year)

# create a column with boolean values to state if the player appears active in fide or not
modified_players_list['Fide Active'] = modified_players_list['National Rank Active'].map( lambda rank_active: rank_active > 0)

col_toshow = [
        'National Name',
        'Fide Name',
        'National Elo', 
        'std', 
        'rapid', 
        'blitz',
        'Age',
        'Birthday',
        'Eso Fee', 
        'Fide Active', 
        'Gender',
        'National ID',
        'Fide ID',
        'Federation',
        'National Title',
        ]
table_toshow = pd.concat([modified_players_list[col] for col in col_toshow], axis=1)
table_toshow.head()

,National Name,Fide Name,National Elo,std,rapid,blitz,Age,Birthday,Eso Fee,Fide Active,Gender,National ID,Fide ID,Federation,National Title
0,ΜΑΪΕΡ ΧΑΝΣ - ΚΡΙΣΤΙΑΝ - ΜΙΧΑΕΛ,0,2336,0,0,0,65,1959-01-01,0,False,M,36055,"<a href=""http://ratings.fide.com/card.phtml?ev...",0,0
1,ΔΡΑΚΟΠΟΥΛΟΣ ΠΟΛΥΧΡΟΝΗΣ - ΠΑΝΑΓΙΩΤΗΣ,Drakopoulos Polihronis,1961,2114,2037,1977,26,1998-05-01,OK,True,M,30932,"<a href=""http://ratings.fide.com/card.phtml?ev...",GRE,0
2,ΤΣΑΓΚΑΡΟΠΟΥΛΟΣ ΣΠΥΡΙΔΩΝ,Tsagkaropoulos Spyridon,1949,2068,1999,1991,35,1989-02-01,OK,True,M,29392,"<a href=""http://ratings.fide.com/card.phtml?ev...",GRE,ΒΚ
3,ΣΠΥΡΑΚΟΠΟΥΛΟΣ ΙΩΑΝΝΗΣ,Spirakopoulos Ioannis,1947,2035,1998,2009,53,1971-12-01,OK,True,M,9345,"<a href=""http://ratings.fide.com/card.phtml?ev...",GRE,ΟΜ
4,ΛΙΑΣΚΟΣ ΛΕΩΝΙΔΑΣ,Liaskos Leonidas,1929,2007,2011,2087,31,1993-03-01,OK,True,M,25836,"<a href=""http://ratings.fide.com/card.phtml?ev...",GRE,ΑΚ


In [8]:
pd.to_datetime('01/' + modified_players_list['Date of Birth'], dayfirst=True)

0     1959-01-01
1     1998-05-01
2     1989-02-01
3     1971-12-01
4     1993-03-01
         ...    
482   2002-03-01
483   2004-12-01
484   2012-08-01
485   2014-03-01
486   2016-03-01
Name: Date of Birth, Length: 487, dtype: datetime64[ns]

## apply restrictions to the table

Restrictions could be the following:
- Gender
- Eso fee
- sort by std or eso rating
- active status
- age

In [4]:
# table to present
col_toshow = [
       'National Name',
        'National Elo', 
        'std', 
        'rapid', 
        'blitz',
        'Age', 
        'Eso Fee', 
        'Fide Active Player', 
        'Gender',
        'National ID',
        'Fide ID hyperlink' 
        ]
table_toshow = pd.concat([modified_players_list[col] for col in col_toshow], axis=1)

table_toshow.to_pickle('table_toshow.pkl')

table_toshow.head().style

,National Name,National Elo,std,rapid,blitz,Age,Eso Fee,Fide Active Player,Gender,National ID,Fide ID hyperlink
0,ΜΑΪΕΡ ΧΑΝΣ - ΚΡΙΣΤΙΑΝ - ΜΙΧΑΕΛ,2336,0,0,0,65,0,False,M,36055,0
1,ΔΡΑΚΟΠΟΥΛΟΣ ΠΟΛΥΧΡΟΝΗΣ - ΠΑΝΑΓΙΩΤΗΣ,1961,2114,2037,1977,26,OK,True,M,30932,4232984
2,ΤΣΑΓΚΑΡΟΠΟΥΛΟΣ ΣΠΥΡΙΔΩΝ,1949,2068,1999,1991,35,OK,True,M,29392,4231813
3,ΣΠΥΡΑΚΟΠΟΥΛΟΣ ΙΩΑΝΝΗΣ,1947,2035,1998,2009,53,OK,True,M,9345,4204123
4,ΛΙΑΣΚΟΣ ΛΕΩΝΙΔΑΣ,1929,2007,2011,2087,31,OK,True,M,25836,4211421


In [5]:
def restrictions(
                 table,
                 elo_type = 'std', 
                 elo_min = 0,
                 elo_max = float('inf'),
                 age_min = 0,
                 age_max = float('inf'),
                 gender = None, 
                 eso_fee = None, 
                 fide_active = None
                  ):

    restrictions = []
    
    if elo_min >0 or elo_max < float('inf'):
        elo_restriction = ( (table_toshow[elo_type] >= elo_min) & (table_toshow[elo_type] <= elo_max) )
        restrictions.append(elo_restriction)

    if age_min >0 or age_max < float('inf'):
        age_restriction = ( (table_toshow['Age']>=age_min ) & (table_toshow['Age'] <= age_max) )
        restrictions.append(age_restriction)
    
    if fide_active!= None and fide_active in [True, False]:
        fide_active_restriction =  table_toshow['Fide Active Player'] == fide_active
        restrictions.append(fide_active_restriction)

    if eso_fee != None and eso_fee in ['OK', '0']:
        eso_fee_restriction = table_toshow['Eso Fee'] == eso_fee
        restrictions.append(eso_fee_restriction)
    
    if gender!= None and gender in 'MF':
        gender_restriction = table_toshow['Gender'] == gender
        restrictions.append(gender_restriction)

    if len(restrictions) == 0:
        return table
    
    # continue if there are some restrictions to apply
    all_restrictions = restrictions[0]

    # combine all restrictions with the logical and operator &
    # I haven't found a way to do that by a built in function
    for res in restrictions:
        all_restrictions = all_restrictions & res    
    
    return table[all_restrictions].sort_values(by=['std','National Elo','Age','National ID'], ascending= False)



In [8]:
restrictions(
             table_toshow,   
             elo_min = 1100,
             age_max= 30,
            #  gender = None, 
            #  fide_active = True,
             eso_fee= None
             ).style

,National Name,National Elo,std,rapid,blitz,Age,Eso Fee,Fide Active Player,Gender,National ID,Fide ID hyperlink
1,ΔΡΑΚΟΠΟΥΛΟΣ ΠΟΛΥΧΡΟΝΗΣ - ΠΑΝΑΓΙΩΤΗΣ,1961,2114,2037,1977,26,OK,True,M,30932,4232984
11,ΡΗΓΑΤΟΣ ΠΑΝΑΓΙΩΤΗΣ,1818,1972,1917,1964,30,OK,True,M,31687,4228146
8,ΛΑΣΠΑΣ ΠΑΝΑΓΙΩΤΗΣ,1843,1966,1906,1916,25,0,False,M,33571,4243102
16,ΚΑΛΟΦΩΝΟΣ ΠΑΝΑΓΙΩΤΗΣ,1745,1892,1959,1927,25,OK,True,M,31682,4233824
22,ΛΙΑΠΗ ΖΕΝΕΜΠΙΣΗ ΕΙΡΗΝΗ,1620,1842,0,1782,26,0,False,F,31828,4234120
37,ΜΑΡΓΑΡΙΤΗΣ ΙΩΑΝΝΗΣ,1517,1842,0,1809,26,OK,True,M,29091,4234219
21,ΚΟΥΤΣΙΔΗΣ ΑΧΙΛΛΕΥΣ,1637,1837,1828,1794,25,0,False,M,33312,4243080
28,ΚΑΡΑΓΙΑΝΝΗΣ ΓΕΩΡΓΙΟΣ,1577,1804,0,0,24,0,False,M,32316,4250478
38,ΛΙΑΠΗ ΖΕΝΕΜΠΙΣΗ ΦΙΛΙΠΠΑ - ΑΡΕΤΗ,1517,1791,0,1718,25,OK,True,F,31827,4246489
45,ΚΑΥΚΑΛΑΣ ΝΙΚΟΛΑΟΣ,1477,1725,0,0,24,0,False,M,30975,4250451


### Search Player by name

In [9]:
import re
your_name = 'μαρκι'
your_name = your_name.upper()
# players_names_national = list(modified_players_list['National Name'])
row_index_match = [i for i in modified_players_list['National Name'].index  if re.search(your_name, modified_players_list['National Name'][i])]
table_toshow.iloc[row_index_match]

,National Name,National Elo,std,rapid,blitz,Age,Eso Fee,Fide Active Player,Gender,National ID,Fide ID hyperlink
35,ΜΑΡΚΙΔΟΥ ΧΡΙΣΤΙΑΝΝΑ,1554,1675,1866,1725,47,0,True,F,37944,"<a href=""http://ratings.fide.com/card.phtml?ev..."


## Statistics

In [7]:
# Groupin by 
modified_players_list.groupby(['Gender', 'Age']).apply(lambda df: df.head())

C:\Users\USER\AppData\Local\Temp\ipykernel_14556\419794589.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_table.groupby(['Gender', 'Age']).apply(lambda df: df.head())


National Name     Fide name  National Elo  std  \
Gender Age                                                                   
F      8   438      ΦΟΥΝΤΟΥΛΑΚΗ ΣΤΑΥΡΟΥΛΑ             0             0    0   
       9   363      ΣΤΑΥΡΟΠΟΥΛΟΥ ΣΤΕΦΑΝΙΑ             0             0    0   
           405     ΤΣΙΑΚΟΠΟΥΛΟΥ ΑΛΕΞΑΝΔΡΑ             0             0    0   
       10  465               ΝΑΣΗ ΕΥΔΟΚΙΑ  Nasi Evdokia             0    0   
       11  357                ΜΠΑΘΑ ΑΘΗΝΑ  Batha Athina             0    0   
...                                   ...           ...           ...  ...   
M      92  31          ΔΕΣΙΠΡΗΣ ΔΗΜΗΤΡΙΟΣ             0          1570    0   
       98  175  ΧΑΤΖΗΠΑΝΑΓΙΩΤΟΥ ΔΗΜΗΤΡΙΟΣ             0          1000    0   
       99  46      ΚΟΥΡΟΥΒΑΝΗΣ ΠΑΝΑΓΙΩΤΗΣ             0          1470    0   
           83            ΑΚΡΙΤΑΣ ΘΕΟΔΩΡΟΣ             0          1235    0   
       101 55        ΜΑΡΑΛΕΤΟΣ ΑΡΙΣΤΕΙΔΗΣ             0          1390    0   

                rapid  blitz  Age Eso Fee  Fide Active Player Gender  ...  \
Gender Age                                                            ...   
F      8   438      0      0    8      OK               False      F  ...   
       9   363      0      0    9       0               False      F  ...   
           405      0      0    9       0               False      F  ...   
       10  465      0      0   10      OK               False      F  ...   
       11  357      0      0   11      OK               False      F  ...   
...               ...    ...  ...     ...                 ...    ...  ...   
M      92  31       0      0   92       0               False      M  ...   
       98  175      0      0   98       0               False      M  ...   
       99  46       0      0   99       0               False      M  ...   
           83       0      0   99       0               False      M  ...   
       101 55       0      0  101       0               False      M  ...   

                National ID   Fide ID  Date of Birth National Title  \
Gender Age                                                            
F      8   438        59541         0        01/2016              0   
       9   363        59149         0        02/2015              0   
           405        59145         0        06/2015              0   
       10  465        60203  42186285        08/2014              0   
       11  357        59544  42182654        08/2013              0   
...                     ...       ...            ...            ...   
M      92  31         10202         0        06/1932             ΒΚ   
       98  175         9846         0        01/1926             ΒΚ   
       99  46          9346         0        01/1925             ΒΚ   
           83          9845         0        01/1925             ΒΚ   
       101 55          2857         0        01/1923             ΔΚ   

               Games last 6 months  Outsider Federation National Rank All  \
Gender Age                                                                  
F      8   438                   0         0          0                 0   
       9   363                   0         0          0                 0   
           405                   0         0          0                 0   
       10  465                   0         0          0                 0   
       11  357                   1         0          0                 0   
...                            ...       ...        ...               ...   
M      92  31                    0         0          0                 0   
       98  175                   0         0          0                 0   
       99  46                    0         0          0                 0   
           83                    0         0          0                 0   
       101 55                    0         0          0                 0   

                National Rank Active  Birthday parsed  
Gender Age                   